In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 40 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 3.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595717 sha256=a68a42815b3941317c8d89ae522be0e1ab7f5269f844cc0d073be01c2ef6d741
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 3.2 MB/s 
     |████████████████████████████████| 754 kB 3.1 MB/s 
     |████████████████████████████████| 3.0 MB 52.7 MB/s 
     |████████████████████████████████| 895 kB 71.9 MB/s 


In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7r9m6436
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-7r9m6436
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=e27d56f4dce3c6e3678d35c546b33e3348a267a904f56681792d61a683544257
  Stored in directory: /tmp/pip-ephem-wheel-cache-yv5cuixz/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
import pandas as pd

In [14]:
device = torch.device("cuda")

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

#infer

In [15]:
model = torch.load('/content/drive/MyDrive/공모전/금융/yogiyo_review_model.pt')

RuntimeError: ignored

In [6]:
unseen_test = pd.read_csv('/content/drive/MyDrive/공모전/tmp/강남면류_topic_label.csv')

NameError: ignored

In [ ]:
unseen_test.head()

In [ ]:
del unseen_test['Unnamed: 0']

In [ ]:
dataset2 = unseen_test

In [ ]:
unseen_test = unseen_test[['review']]

In [ ]:
unseen_test['label1'] = 9

In [ ]:
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)
test_output = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  test_output.append(out.tolist())

In [ ]:
np.array(test_output)[:,:,0]

In [ ]:
dataset2['sentimental_score_negative'] = np.array(test_output)[:,:,0]

In [ ]:
dataset2['sentimental_score_positive'] = np.array(test_output)[:,:,1]

In [ ]:
dataset2

In [ ]:
dataset2.to_csv('/content/drive/MyDrive/공모전/금융/sentimental_score_kobert_강남_면류.csv', encoding = 'utf-8')